In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install shap==0.39

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for shap: filename=shap-0.39.0-cp310-cp310-linux_x86_64.whl size=497648 sha256=38f760e9c1586eb502c389d4764b8115a49f5ad5c1ec757dd6fa391dceb59fc4
  Stored in directory: /root/.cache/pip/wheels/d4/ef/8d/78a07e01b86b1a0363626186f072e01683c10335e2eb89e337
Successfully built shap
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: shap
    Found existing installation: shap 0.46.0
    Uninstalling shap-0.46.0:
      Successfully uninstalled shap-0.46.0


In [4]:
def get_labels_lst(shap_values,predicted_label): #Ret ve ihlal durumuna etki eden token ve shapley değerini list veri tipinde döndüren fonksiyon
    ihlal_lst=[]
    ret_lst=[]
    for value, token in zip(shap_values.values[0],shap_values.data[0]):
        #print(value, token)
        val_token={}
        #print(value,token)
        if(predicted_label==1): #İhlalse
            val_token["Value"]=value[1]
            val_token["Token"]=token
            ihlal_lst.append(val_token)
        else: #Retse
            val_token["Value"]=value[0]
            val_token["Token"]=token
            ret_lst.append(val_token)
    if(predicted_label==1):
        return ihlal_lst
    else:
        return ret_lst
import math
def get_important_parts(threshold_ratio,shap_values,predicted_label): #ilgili etikete göre önemli kısımları belli bir orana göre seçen fonksiyon
    lst=get_labels_lst(shap_values,predicted_label)
    reference_val=lst[0]["Value"]
    print(reference_val)
    important_parts={}
    for val_tokens in lst:
        if (math.isclose(val_tokens['Value'],reference_val)):
            if(reference_val in important_parts.keys()):
                token_lst=important_parts[reference_val]
                token_lst.append(val_tokens["Token"])
                important_parts[reference_val]=token_lst
            else:
                important_parts[reference_val]=[val_tokens["Token"]]
                #print(important_parts)
        else:
            reference_val=val_tokens["Value"]
            important_parts[reference_val]=[val_tokens["Token"]]
    keys = list(important_parts.keys())
    keys.sort()
    threshold=keys[int(len(keys)*threshold_ratio)]
    print(threshold)
    sorted_important_parts = {i: ''.join(important_parts[i]) for i in keys}
    result_str=""
    for values,tokens in important_parts.items(): # Belli bir threshold değerinin üzerindeki kısımları almayı sağlayan koşul
        if(values>threshold):
            result_str+= "".join(tokens)
    return result_str #,important_parts,sorted_important_parts

In [5]:
import pickle
import numpy as np
with open("/content/drive/MyDrive/Graduation Project - CSE4297/Dataset for ML Project/Hugging Face Dataset Format/90_5_5_binary_hugging_face_dataset.pkl","rb") as f:
    splitted_dataset=pickle.load(f)

In [6]:
length_test_set=len(splitted_dataset['test']['text'])
length_test_set

665

In [7]:
import shap
import transformers
import torch
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TextClassificationPipeline)

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
import shap
import numpy as np
import json

# Fix for deprecated np.bool
np.bool = bool
np.int=int
np.object = object
np.float=float
# Define checkpoint and load model/tokenizer
checkpoint = "/content/drive/MyDrive/Graduation Project - CSE4297/Dataset for ML Project/Training Code for DL/aym-bert-turkish-base/checkpoint-748"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint).eval()
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create pipeline
pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    top_k=None,
    max_length=512,
    truncation=True,
    device=0
)

# SHAP masker and explainer
masker = shap.maskers.Text(tokenizer, mask_token=tokenizer.pad_token)
explainer = shap.Explainer(pipe, masker=masker)

# Test dataset
important_strs_dict = {}
length_test_set = len(splitted_dataset['test']['text'])

for index in range(min(15, length_test_set)):  # Adjust test range
    text = splitted_dataset['test']['text'][index]
    tokens = tokenizer.tokenize(text)
    text = tokenizer.convert_tokens_to_string(tokens)
    true_label = splitted_dataset['test']['labels'][index]

    print("Modele verilen metin:", text)
    print("Doğru etiket:", true_label)

    prediction = pipe([text])
    predicted_label = 1 if prediction[0][1]['score'] > prediction[0][0]['score'] else 0
    print("Modelin tahmini:", "İhlal" if predicted_label == 1 else "Ret")

    correct_prediction = predicted_label == true_label

    # SHAP explanation
    shap_values = explainer([text])
    shap.plots.text(shap_values)


Output hidden; open in https://colab.research.google.com to view.